<a href="https://colab.research.google.com/github/jaxayprajapati/Machine-Learning/blob/main/SklearnPipelines.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**What is Scikit Learn Pipelines**


*   Pipelines chains together multiple steps so that the output of each step is used as inpt to the next step
*   Pipelines makes it easy to apply the same preprocessing to train and test!


I will take vey simple and famous titanic dataset



In [4]:
# import required libraries
import pandas as pd
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.tree import DecisionTreeClassifier

In [6]:
# import data
url = 'https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv'
data = pd.read_csv(url)
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


Let's plan our pipeline



1.   Impute missing values
2.   OneHotEncoding
3.   Scaling
4.   Feature selection
5.   Model creation



In [7]:
data.drop(columns=['PassengerId', 'Name', 'Ticket', 'Cabin'], inplace=True)

In [35]:
# Train - Test split
X = data.drop('Survived', axis=1)
y = data['Survived']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [9]:
X_train.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
331,1,male,45.5,0,0,28.5000,S
733,2,male,23.0,0,0,13.0000,S
382,3,male,32.0,0,0,7.9250,S
704,3,male,26.0,1,0,7.8542,S
813,3,female,6.0,4,2,31.2750,S


In [10]:
y_train.head()

331    0
733    0
382    0
704    0
813    0
Name: Survived, dtype: int64

In [27]:
# imputation transfomer
imputation_transformer = ColumnTransformer([
    ('impute_age', SimpleImputer(),[2]),
    ('imput_embarked', SimpleImputer(strategy='most_frequent'),[6])
], remainder='passthrough')

In [28]:
# onehotencoding
one_hot_transformer = ColumnTransformer([
    ('ohe_gender_embarked', OneHotEncoder(sparse_output=False, handle_unknown='ignore'), [1,6])
],remainder='passthrough')

In [29]:
# Scaling
scaler = ColumnTransformer([
    ('scale', MinMaxScaler(),slice(0,10))
])

In [30]:
# Feature selection
selected_features = SelectKBest(score_func=chi2, k=8)

In [31]:
# Model creation
model = DecisionTreeClassifier()

**Pipeline creation**

In [32]:
pipeline = Pipeline([
    ('imputation_transformer',imputation_transformer),
    ('one_hot_transformer',one_hot_transformer),
    ('scaler',scaler),
    ('selected_features', selected_features),
    ('model', model)
])

In [33]:
# Pass out data in pipe line
pipeline.fit(X_train, y_train)

Pipeline(steps=[('imputation_transformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('impute_age', SimpleImputer(),
                                                  [2]),
                                                 ('imput_embarked',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [6])])),
                ('one_hot_transformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe_gender_embarked',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  [1, 6])])),
                ('scaler',
                 ColumnTransformer(transformers=[('scale', MinMaxScaler(),
                                                  slice(0, 10, None))])),
                ('selected_features',
                 SelectKBest(k=8,
                             score_func=<function chi2 at 0x7b6499578820>)),
                ('model', DecisionTreeClassifier())])

In [36]:
# Prediction
y_pred = pipeline.predict(X_test)
y_pred

array([1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1,
       0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1,
       0, 0, 0])

In [37]:
# Export Pipe line
import pickle
pickle.dump(pipeline, open('decisionTreeModel.pkl','wb'))

# Prdiction on new data

In [39]:
pipeline_model = pickle.load(open('/content/decisionTreeModel.pkl','rb'))

In [40]:
test_input = np.array([2,'male',20.0,1,1,36.6,'C'],dtype=object).reshape(1,7)

In [41]:
test_input

array([[2, 'male', 20.0, 1, 1, 36.6, 'C']], dtype=object)

In [42]:
pipeline_model.predict(test_input)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but SimpleImputer was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but SimpleImputer was fitted with feature names
  warnings.warn(


array([1])